This notebook will be usefull for us to test the python's scripts

In [47]:
DBG = True
def print_dbg(*args):
    if(DBG):
        print(*args)

In [48]:
def get_var_or_init(var_name, var_shape, scope_name, init_func):
    try:
        with tf.variable_scope(scope_name) as scope:
            scope.reuse_variables()
            var = tf.get_variable(var_name, var_shape)
    except Exception as e:
        print("Error", e)
        var = tf.get_variable(var_name, var_shape, initializer=init_func)
    return var

In [49]:
# This function is actually better than the functions present in:
# Here: https://stackoverflow.com/questions/40891533/valueerror-variable-weights-does-not-exist-or-was-not-created-with-tf-get-vari
# And Here: https://stackoverflow.com/questions/38545362/tensorflow-variable-scope-reuse-if-variable-exists
def init_or_fetch(var_name, var_shape, scope_name, init_values, mode="auto"):
    if(mode=="auto"):
        with tf.variable_scope(scope_name, reuse=tf.AUTO_REUSE) as scope:
            var = tf.get_variable(var_name, var_shape)#, initializer = init_values)
            #print(var)
            return var
    elif(mode=="init"):
        '''initialize variables within variable scope_name.'''
        with tf.variable_scope(scope_name, reuse=None) as scope:
            var = tf.get_variable(var_name, var_shape, initializer = init_values)
            scope.reuse_variables()
            return var
    elif(mode=="fetch"):
        '''fetch variables within variable scope_name'''
        with tf.variable_scope(scope_name, reuse=True):
            var = tf.get_variable(var_name)
            return var
    else:
        raise Exception("mode must be either 'auto', 'init' or 'fetch', {} is an invalid mode".format(mode))

In [50]:
with tf.variable_scope('gen') as scope:
        w = init_or_fetch('wasp', (2,3,4), 'gen', tf.truncated_normal_initializer(stddev=0.002))
print(w)

<tf.Variable 'gen/gen/wasp:0' shape=(2, 3, 4) dtype=float32_ref>


ops.py

In [51]:
import math
import numpy as np 
import tensorflow as tf

from tensorflow.python.framework import ops

from src.utils import *

def batch_norm(x, name="batch_norm", train=True, epsilon=1e-5, momentum = 0.9):
    return tf.contrib.layers.batch_norm(x, decay=momentum, updates_collections=None, epsilon=epsilon, scale=True, scope=name)

def binary_cross_entropy(preds, targets, name=None):
    """Computes binary cross entropy given `preds`.

    For brevity, let `x = `, `z = targets`.  The logistic loss is

        loss(x, z) = - sum_i (x[i] * log(z[i]) + (1 - x[i]) * log(1 - z[i]))

    Args:
        preds: A `Tensor` of type `float32` or `float64`.
        targets: A `Tensor` of the same type and shape as `preds`.
    """
    eps = 1e-12
    with ops.op_scope([preds, targets], name, "bce_loss") as name:
        preds = ops.convert_to_tensor(preds, name="preds")
        targets = ops.convert_to_tensor(targets, name="targets")
        return tf.reduce_mean(-(targets * tf.log(preds + eps) +
                              (1. - targets) * tf.log(1. - preds + eps)))

def conv_cond_concat(x, y):
    """Concatenate conditioning vector on feature map axis."""
    x_shapes = x.get_shape()
    y_shapes = y.get_shape()
    return tf.concat([x, y*tf.ones([x_shapes[0], x_shapes[1], x_shapes[2], y_shapes[3]])], 3)

def conv2d(input_, output_dim, k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02, name="conv2d"):
    var_shape = [k_h, k_w, input_.get_shape()[-1], output_dim]
    with tf.variable_scope(name) as scope:
#         scope.reuse_variables()
#         w = tf.get_variable('w', [k_h, k_w, input_.get_shape()[-1], output_dim],
#                             initializer=tf.truncated_normal_initializer(stddev=stddev))
        w = init_or_fetch('w', var_shape, name, tf.truncated_normal_initializer(stddev=stddev))
        conv = tf.nn.conv2d(input_, w, strides=[1, d_h, d_w, 1], padding='SAME')

        biases = tf.get_variable('biases', [output_dim], initializer=tf.constant_initializer(0.0))
        conv = tf.reshape(tf.nn.bias_add(conv, biases), conv.get_shape())

        return conv

def deconv2d(input_, output_shape, k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02, name="deconv2d", with_w=True):
    var_shape = [k_h, k_w, output_shape[-1], input_.get_shape()[-1]]
    with tf.variable_scope(name) as scope:
#        scope.reuse_variables()
        # filter : [height, width, output_channels, in_channels]
#        w = tf.get_variable('w', [k_h, k_w, output_shape[-1], input_.get_shape()[-1]],
#                            initializer=tf.random_normal_initializer(stddev=stddev))
        w = init_or_fetch('w', var_shape, name, tf.truncated_normal_initializer(stddev=stddev))
        
        try:
            deconv = tf.nn.conv2d_transpose(input_, w, output_shape=output_shape,
                                strides=[1, d_h, d_w, 1])

        # Support for verisons of TensorFlow before 0.7.0
        except AttributeError:
            deconv = tf.nn.deconv2d(input_, w, output_shape=output_shape,
                                strides=[1, d_h, d_w, 1])

        biases = tf.get_variable('biases', [output_shape[-1]], initializer=tf.constant_initializer(0.0))
        deconv = tf.reshape(tf.nn.bias_add(deconv, biases), deconv.get_shape())

        if with_w:
            return deconv
        else:
            return deconv

def lrelu(x, leak=0.2, name="lrelu"):
    return tf.maximum(x, leak*x)

def linear(input_, output_size, scope=None, stddev=0.02, bias_start=0.0, with_w=False):
    shape = input_.get_shape().as_list()

    with tf.variable_scope(scope or "Linear"):
        matrix = tf.get_variable("Matrix", [shape[1], output_size], tf.float32,
                                 tf.random_normal_initializer(stddev=stddev))
        bias = tf.get_variable("bias", [output_size],
            initializer=tf.constant_initializer(bias_start))
        if with_w:
            return tf.matmul(input_, matrix) + bias, matrix, bias
        else:
            return tf.matmul(input_, matrix) + bias
            
def residual_block(input_d, size, type):
    rb = input_d
    rbCount = 4
    for i in range(rbCount):
        rb = conv2d(rb, input_d.shape[-1].value, d_h=1, d_w=1, name='g_rb_' + str(size) + '_conv_rb' + str(type) + '_' + str(i))
        rb = batch_norm(rb, name="g_rb_bn_" + str(size) + "_" + str(type) + "_" + str(i))
        rb = tf.nn.relu(rb)
    rb_sum = tf.add(input_d, rb, name='g_rb_' + str(size) + '_add_rb' + str(type))
    return tf.nn.relu(rb_sum)

def multi_residual_block(input_d, size):
    rbCount = 2
    rb = input_d
    for i in range(rbCount):
        rb = residual_block(rb, size, i)
    return rb


Model.py

In [52]:
# -*- coding: utf-8 -*- 
import tensorflow as tf
from glob import glob
import os
import time
import numpy as np
import random

#from src.ops import *
from src.utils import *

class ScaleGan(object):
    def __init__(self, sess, dataset_name, origin_size=64, img_size=256):
        self.dataset_name = dataset_name
        self.batch_size = 50
        self.checkpoint_dir = './checkpoint'
        self.img_dim = 3 # image file color channel
        self.conv_dim = 64
        self.sess = sess
        self.img_size = img_size 
        self.L1_lambda = 100
        self.LAMBDA = 10 # Gradient penalty lambda hyperparameter
        self.origin_size = origin_size
        self.build_model()

    def build_model(self):
        print("build model - input_img", [self.batch_size, self.img_size, self.img_size, self.img_dim*2])
        # Inserts a placeholder for a tensor that will be always fed.
        # self.input_img = tf.compat.v1.placeholder(tf.float32, 
        self.input_img = tf.placeholder(tf.float32, 
            [self.batch_size, self.img_size, self.img_size, self.img_dim*2],
            name='input_A_and_B_images')
        sizes = []
        size = self.origin_size
        print("build_model - origin_size", self.origin_size)
        while(size < self.img_size):
            print("build_model - img_size", self.img_size, size)
            sizes.append(size)
            size = size*2
        if size != self.img_size:
            print("size error when building model. origin size is " 
                    + str(self.origin_size) + " and target image size is " + str(self.img_size))
            return
        
        # --- SPLIT SECTION ----------------------------------------------------------------
        # In this section image is splited into 2: fake (low-resolution) and real (ground truth)
        print("build_model - size", size)
        # A is sample, B is ground truth,
        self.real_A = []
        A = self.input_img[:, :, :, :self.img_dim]
        for size in sizes:
            #self.real_A.append(tf.compat.v1.image.resize_images(A, (size, size)))
            self.real_A.append(tf.image.resize_images(A, (size, size)))
        self.real_A.append(A)
        self.real_B = []
        B = self.input_img[:, :, :, self.img_dim:2*self.img_dim]
        for size in sizes:
            #self.real_B.append(tf.compat.v1.image.resize_images(B, (size, size)))
            self.real_B.append(tf.image.resize_images(B, (size, size)))
        self.real_B.append(B)

        # ...................................................................................
        
        self.fake_B = self.generator(self.real_A[-1])
        self.fake_sample = self.sampler(self.real_A[-1])

        self.real_AB = []
        for i in range(len(self.real_A)):
            self.real_AB.append(tf.concat([self.real_A[i], self.real_B[i]], 3))
        self.fake_AB = []
        for i in range(len(self.real_A)):
            self.fake_AB.append(tf.concat([self.real_A[i], self.fake_B[i]], 3))
        self.both_BB = []
        for i in range(len(self.real_B)):
            self.both_BB.append(tf.concat([self.real_B[i], self.fake_B[i]], 3))

        self.D_real_AB, self.D_real_AB_logits = self.discriminator(self.real_AB, name="discriminator")
#         self.D_real_AB, self.D_real_AB_logits = self.discriminator(self.real_AB, name="discriminator", reuse=True)
        self.D_fake_AB, self.D_fake_AB_logits = self.discriminator(self.fake_AB, name="discriminator", reuse=True)
        self.D_both_BB, self.D_both_BB_logits = self.discriminator(self.both_BB, name="discriminator", reuse=True)

        self.D_sum_real_AB = []
        for i in range(len(self.D_real_AB)):
            self.D_sum_real_AB.append(tf.summary.histogram("d_real_AB_" + str(i), self.D_real_AB[i]))
        self.D_sum_fake_AB = []
        for i in range(len(self.D_fake_AB)):
            self.D_sum_fake_AB.append(tf.summary.histogram("d_fake_AB_" + str(i), self.D_fake_AB[i]))
        self.D_sum_both_BB = []
        for i in range(len(self.D_both_BB)):
            self.D_sum_both_BB.append(tf.summary.histogram("d_both_BB_" + str(i), self.D_both_BB[i]))
        self.fake_B_sum = []
        for i in range(len(self.fake_B)):
            self.fake_B_sum.append(tf.summary.image("fake_B_" + str(i), self.fake_B[i]))

        self.d_loss_real = []
        for i in range(len(self.D_real_AB)):
            self.d_loss_real.append(tf.reduce_mean(
                    tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D_real_AB_logits[i], labels=tf.ones_like(self.D_real_AB[i]))))
        self.d_loss_fake = []
        for i in range(len(self.D_fake_AB)):
            self.d_loss_fake.append(tf.reduce_mean(
                    tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D_fake_AB_logits[i], labels=tf.zeros_like(self.D_fake_AB[i]))))
        self.d_loss_both = []
        for i in range(len(self.D_both_BB)):
            self.d_loss_both.append(tf.reduce_mean(
                    tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D_both_BB_logits[i], labels=tf.zeros_like(self.D_both_BB[i]))))
        self.g_loss = []
        for i in range(len(self.D_both_BB)):
            if i == 0:
                self.g_loss.append(
                    tf.reduce_mean(
                        tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D_both_BB_logits[i], labels=tf.ones_like(self.D_both_BB[i])))
                    + self.L1_lambda * tf.reduce_mean(tf.abs(self.real_B[i] - self.fake_B[i]))
                    + tf.reduce_mean(
                        tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D_fake_AB_logits[i], labels=tf.ones_like(self.D_fake_AB[i])))
                )
            else:
                self.g_loss.append(
                    self.L1_lambda * tf.reduce_mean(tf.abs(self.real_B[i] - self.fake_B[i]))
                    + tf.reduce_mean(
                        tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D_fake_AB_logits[i], labels=tf.ones_like(self.D_fake_AB[i])))
                )
        
        self.d_loss_real_sum = []
        for i in range(len(self.d_loss_real)):
            self.d_loss_real_sum.append(tf.summary.scalar("d_loss_real_" + str(i), self.d_loss_real[i]))
        self.d_loss_fake_sum = []
        for i in range(len(self.d_loss_fake)):
            self.d_loss_fake_sum.append(tf.summary.scalar("d_loss_fake_" + str(i), self.d_loss_fake[i]))
        self.d_loss_both_sum = []
        for i in range(len(self.d_loss_both)):
            self.d_loss_both_sum.append(tf.summary.scalar("d_loss_both_" + str(i), self.d_loss_both[i]))

        self.d_loss = []
        for i in range(len(self.d_loss_real)):
            if i == 0:
                self.d_loss.append(self.d_loss_real[i] + self.d_loss_fake[i] + self.d_loss_both[i])
            else:
                self.d_loss.append(self.d_loss_real[i] + self.d_loss_fake[i])
        self.d_loss_sum = []
        for i in range(len(self.d_loss)):
            self.d_loss_sum.append(tf.summary.scalar("d_loss_" + str(i), self.d_loss[i]))
        self.g_loss_sum = []
        for i in range(len(self.d_loss)):
            self.g_loss_sum.append(tf.summary.scalar("g_loss_" + str(i), self.g_loss[i]))
            
        t_vars = tf.trainable_variables()
        self.d_vars = []
        for var in t_vars:
            if 'd_' in var.name:
                self.d_vars.append(var)
        self.g_vars = []
        for var in t_vars:
            if 'g_' in var.name:
                self.g_vars.append(var)

        self.saver = tf.train.Saver()       
        
    def load(self, checkpoint_dir):
        print(" [*] Reading checkpoint...")

        model_dir = "%s_%s_%s" % (self.dataset_name, self.batch_size, self.img_size)
        checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            return True
        else:
            return False
            
    def save(self, checkpoint_dir, step):
        model_name = "pix2pix.model"
        model_dir = "%s_%s_%s" % (self.dataset_name, self.batch_size, self.img_size)
        checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess, os.path.join(checkpoint_dir, model_name), global_step=step)

    def load_random_samples(self):
        val_dataset = './datasets/{}/val/*.jpg'.format(self.dataset_name)
        val_data = glob(val_dataset)
        #print_dbg(val_dataset, val_data)
        data = np.random.choice(val_data, self.batch_size)
        sample = [load_data(sample_file, self.img_size, self.img_size + int(self.img_size/8)) for sample_file in data]
        sample_images = np.array(sample).astype(np.float32)
        return sample_images
        
    def sample_model(self, sample_dir, epoch):
        sample_images = self.load_random_samples()
        samples, d_loss, g_loss = self.sess.run(
            [self.fake_sample, self.d_loss[-1], self.g_loss[-1]], feed_dict={self.input_img: sample_images}
        )
        save_merge_images(samples, [self.batch_size, 1], './{}/train_{:02d}.png'.format(sample_dir, epoch))
        print("[Sample] d_loss: {:.8f}, g_loss: {:.8f}".format(d_loss, g_loss))
    
    def train(self, args):
        #TODO: Fix a "here" error
#         with tf.variable_scope(tf.get_variable_scope(), reuse=False) as scope:
#             assert tf.get_variable_scope().reuse == True
        d_optim = []
        for i in range(len(self.d_loss)):
            d_optim.append(tf.train.AdamOptimizer(args.lr, beta1=args.beta1).minimize(self.d_loss[i], var_list=self.d_vars))
        g_optim = []
        for i in range(len(self.g_loss)):
            g_optim.append(tf.train.AdamOptimizer(args.lr, beta1=args.beta1).minimize(self.g_loss[i], var_list=self.g_vars))

        init_op = tf.global_variables_initializer()
        self.sess.run(init_op)

        g_sum_list = []
        for i in range(len(self.D_sum_fake_AB)):
            g_sum_list.append(self.D_sum_fake_AB[i])
            g_sum_list.append(self.D_sum_both_BB[i])
            g_sum_list.append(self.fake_B_sum[i])
            g_sum_list.append(self.d_loss_fake_sum[i])
            g_sum_list.append(self.d_loss_both_sum[i])
            g_sum_list.append(self.g_loss_sum[i])
        self.g_sum = tf.summary.merge(g_sum_list)
        d_sum_list = []
        for i in range(len(self.D_sum_real_AB)):
            d_sum_list.append(self.D_sum_real_AB[i])
            d_sum_list.append(self.d_loss_real_sum[i])
            d_sum_list.append(self.d_loss_sum[i])
        self.d_sum = tf.summary.merge(d_sum_list)

        self.writer = tf.summary.FileWriter("./logs", self.sess.graph)
        counter = 1
        start_time = time.time()
        if self.load(self.checkpoint_dir):
            print(" [*] Load SUCCESS")
        else:
            print(" [!] Load failed...")
            
        for epoch in range(args.epoch):
            data = glob('./datasets/{}/train/*.jpg'.format(self.dataset_name))
            random.shuffle(data)
            batch_idxs = min(len(data), args.train_size) // self.batch_size
            for idx in range(0, batch_idxs):
                batch_files = data[idx*self.batch_size:(idx+1)*self.batch_size]
                batch = [load_data(batch_file, self.img_size, self.img_size+int(self.img_size/8)) for batch_file in batch_files]
                batch_images = np.array(batch).astype(np.float32)
                for i in range(len(d_optim)):
                    _, summary_str = self.sess.run([d_optim[i], self.d_sum], feed_dict={self.input_img: batch_images})
                    self.writer.add_summary(summary_str, counter)
                    _, summary_str = self.sess.run([g_optim[i], self.g_sum], feed_dict={self.input_img: batch_images})
                    self.writer.add_summary(summary_str, counter)
                    _, summary_str = self.sess.run([g_optim[i], self.g_sum], feed_dict={self.input_img: batch_images})
                    self.writer.add_summary(summary_str, counter)
                errD = ""
                errG = ""
                for i in range(len(self.d_loss)):
                    result = self.d_loss[i].eval({self.input_img: batch_images})
                    errD += "{:.4f},".format(result)
                    result = self.g_loss[i].eval({self.input_img: batch_images})
                    errG += "{:.4f},".format(result)
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, d_loss: [%s], g_loss: [%s]" \
                        % (epoch, idx, batch_idxs, time.time() - start_time, errD[:-1], errG[:-1]))
                if np.mod(counter, 500) == 2:
                    self.save(args.checkpoint_dir, counter)
                        
            self.sample_model(args.sample_dir, epoch)


    def discriminator(self, img, name, reuse=False):
        with tf.variable_scope(name):
            if reuse:
                tf.get_variable_scope().reuse_variables()
            else:
                assert tf.get_variable_scope().reuse == False
            D = []
            D_logits = []
            for i in range(len(img)):
                h = img[i]
                size = h.shape[1].value
                count = 0
                while size > 8:
                    fact = 2**count
                    if fact > 8:
                        fact = 8
                    h = batch_norm(
                        conv2d(h, self.conv_dim*fact, name="d_" + str(i) + "_h" + str(count) + "_conv"),
                        name="d_" + str(i) + "_bn" + str(count))
                    h = lrelu(h)
                    size = h.shape[1].value
                    count += 1
                D.append(tf.nn.sigmoid(h))
                D_logits.append(h)
            return D, D_logits

    def scaleImage(self, img):
        e = conv2d(img, self.conv_dim, name='g_e0_conv')
        size = e.shape[1]
        count = 1
        eList = [e]
        # from img_size to 1x1
        while size > 1:
            fact = 2**count
            print_dbg("size, fact, count", size, fact, count)
            if fact > 8:
                fact = 8
            var_name_1 = "g_e" + str(count) + "_conv"
            var_name_2 = "g_bn_e" + str(count) + "_conv"
            print_dbg("scaleImg - vars", var_name_1, var_name_2)
            e = batch_norm(conv2d(lrelu(e), self.conv_dim*fact, name=var_name_1), name=var_name_2)
            size = e.shape[1].value
            count += 1
            if size == 1:
                break
            eList.append(e)
        #print("eList", eList)
        # e is 1x1
        d = e
        d_for_out = None
        # from 1x1 to origin_size
        while size < self.origin_size:
            size = size*2
            d_for_out = d
            fact = 2**count
            if fact > 8:
                fact = 8
            var_name_1 = "g_d" + str(count) + "_deconv"
            var_name_2 = "g_bn_d" + str(count) + "_deconv"
            print_dbg("first deconv", var_name_1, var_name_2)
            d = deconv2d(tf.nn.relu(d), [self.batch_size, size, size, self.conv_dim*fact], name=var_name_1)
            d = batch_norm(d, name=var_name_2)
            d = tf.concat([d, eList[-1]], 3)
            del eList[-1]
            count -= 1
        # from origin_size to img_size
        fakeB = []
        while size <= self.img_size:
            B = deconv2d(tf.nn.relu(d_for_out), [self.batch_size, size, size, self.img_dim],
                            name="g_d" + str(count) + "_out")
            fakeB.append(tf.nn.tanh(B))
            size = size*2
            count -=1
            rb = multi_residual_block(d, size)
            d_for_out = rb
            fact = 2**count
            if fact > 8:
                fact = 8
            if size < self.img_size:
                print_dbg("second deconv", var_name_1, var_name_2)
                d = deconv2d(tf.nn.relu(rb), [self.batch_size, size, size, self.conv_dim*fact],
                                name="g_d" + str(count) + "_deconv")
                d = batch_norm(d, name="g_bn_d" + str(count) + "_deconv")
                d = tf.concat([d, eList[-1]], 3)
                del eList[-1]
        return fakeB

    def generator(self, img):
        #with tf.compat.v1.variable_scope("generator"):
        with tf.variable_scope("generator") as scope:
#             scope.reuse_variables()
            return self.scaleImage(img)

    def sampler(self, img):
        #with tf.compat.v1.variable_scope("generator") as scope:
        with tf.variable_scope("generator") as scope:
            scope.reuse_variables()
            fakeB = self.scaleImage(img)
            return fakeB[-1]

    def test(self, args):
        init_op = tf.global_variables_initializer()
        self.sess.run(init_op)
        sample_files_all = glob('./datasets/{}/val_test/*.jpg'.format(self.dataset_name))
        
        max_size = 10000
        batch_count = 0
        while len(sample_files_all) > max_size * batch_count:
            endIdx = max_size * (batch_count + 1)
            if (len(sample_files_all) < endIdx):
                endIdx = len(sample_files_all)
            sample_files = sample_files_all[batch_count * max_size : endIdx]
            print("Loading testing images ... from {0} to {1} of total {2}".format(batch_count * max_size, endIdx, len(sample_files_all)))
            batch_count += 1
            sample = [load_data(sample_file, self.img_size, self.img_size+int(self.img_size/8), is_test=True) for sample_file in sample_files]
            sample_images = np.array(sample).astype(np.float32)
            sample_images = [sample_images[i:i+self.batch_size] for i in range(0, len(sample_images), self.batch_size)]
            sample_images = np.array(sample_images)

            if self.load(self.checkpoint_dir):
                print(" [*] Load SUCCESS")
            else:
                print(" [!] Load failed...")
            print("file number: {}".format(len(sample_files)))

            for i, sample_image in enumerate(sample_images):
                idx = i
                fileName = sample_files[idx].split('/')[-1].split('.jpg')[0]
                print("sampling image {}, {} of total {}".format(fileName, idx + (batch_count - 1) * max_size, len(sample_files_all) // self.batch_size))
                samples = self.sess.run(self.fake_sample, feed_dict={self.input_img: sample_image})
                for j in range(self.batch_size):
                    jdx = j+i*self.batch_size
                    save_images(samples[j, :, :, :], './{}/{}.png'.format(args.test_dir, sample_files[jdx].split('/')[-1].split('.jpg')[0]))


2.train_and_test.py

In [53]:
# -*- coding: utf-8 -*- 
import argparse
import os

#from src.model import ScaleGan
import tensorflow as tf

parser = argparse.ArgumentParser(description='')
parser.add_argument('--phase', dest='phase', default='test', help='phase')
parser.add_argument('--dataset_name', dest='dataset_name', default='celeb_train', help='name of the dataset')
parser.add_argument('--epoch', dest='epoch', type=int, default=200, help='# of epoch')
parser.add_argument('--train_size', dest='train_size', type=int, default=500, help='# images used to train')
parser.add_argument('--lr', dest='lr', type=float, default=0.0002, help='initial learning rate for adam')
parser.add_argument('--beta1', dest='beta1', type=float, default=0.5, help='momentum term of adam')
parser.add_argument('--checkpoint_dir', dest='checkpoint_dir', default='./checkpoint', help='models are saved here')
parser.add_argument('--sample_dir', dest='sample_dir', default='./sample', help='sample are saved here')
parser.add_argument('--test_dir', dest='test_dir', default='./test', help='test sample are saved here')
parser.add_argument('--origin_size', dest='origin_size', type=int, default=64, help='original image size')
parser.add_argument('--image_size', dest='image_size', type=int, default=256, help='image size')


# args = parser.parse_args()

def execute(checkpoint_dir, sample_dir, test_dir, dataset_name, origin_size, image_size, phase, all_args):
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    if not os.path.exists(sample_dir):
        os.makedirs(sample_dir)
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)
        
    #with tf.compat.v1.Session() as sess:
    with tf.Session() as sess:
        model = ScaleGan(sess, dataset_name=dataset_name,
                        origin_size=origin_size, img_size=image_size)

        if phase == 'train':
            model.train(all_args)
        else:
            model.test(all_args)

def main(_):
    execute(
        args.checkpoint_dir, args.sample_dir, args.test_dir,
        args.dataset_name, args.origin_size, args.image_size, args.phase, args
    )

if __name__ == '__main__':
    pass
    #tf.app.run()
    #tf.compat.v1.app.run()

In [ ]:
# From README
# `python 2.train_and_test.py --phase train --dataset_name yourData --origin_size 64 --image_size 256`

class ArgClass(object):
    pass

arguments = ArgClass()

setattr(arguments, 'checkpoint_dir',  './checkpoint')
setattr(arguments, 'sample_dir',  './sample')
setattr(arguments, 'test_dir',  './test')
setattr(arguments, 'dataset_name',  'mydata')
setattr(arguments, 'origin_size',  64)
setattr(arguments, 'image_size',  256)
setattr(arguments, 'phase',  'train')
setattr(arguments, 'epoch',  200)
setattr(arguments, 'train_size',  500)
setattr(arguments, 'lr',  0.0002)
setattr(arguments, 'beta1',  0.5)

# https://stackoverflow.com/questions/47296969/valueerror-variable-rnn-basic-rnn-cell-kernel-already-exists-disallowed-did-y
tf.reset_default_graph()
# Related issues:
# https://stackoverflow.com/questions/46056206/tensorflow-value-error-variable-already-exists-disallowed
# https://medium.com/@hideyuki/what-does-variable-reuse-mean-in-tensorflow-40e86535026b
# https://stackoverflow.com/questions/50210785/tf-how-to-solve-valueerror-variable-weight-already-exists-disallowed-did
# https://cs224d.stanford.edu/lectures/CS224d-Lecture7.pdf

execute('./checkpoint', './sample', './test', arguments.dataset_name, 64, 256, 'train', arguments)
#tf.compat.v1.app.run()
tf.app.run()

build model - input_img [50, 256, 256, 6]
build_model - origin_size 64
build_model - img_size 256 64
build_model - img_size 256 128
build_model - size 256
128 2 1
g_e1_conv g_bn_e1_conv
64 4 2
g_e2_conv g_bn_e2_conv
32 8 3
g_e3_conv g_bn_e3_conv
16 16 4
g_e4_conv g_bn_e4_conv
8 32 5
g_e5_conv g_bn_e5_conv
4 64 6
g_e6_conv g_bn_e6_conv
2 128 7
g_e7_conv g_bn_e7_conv
[<tf.Tensor 'generator/g_e0_conv/Reshape:0' shape=(50, 128, 128, 64) dtype=float32>, <tf.Tensor 'generator/g_bn_e1_conv/Identity:0' shape=(50, 64, 64, 128) dtype=float32>, <tf.Tensor 'generator/g_bn_e2_conv/Identity:0' shape=(50, 32, 32, 256) dtype=float32>, <tf.Tensor 'generator/g_bn_e3_conv/Identity:0' shape=(50, 16, 16, 512) dtype=float32>, <tf.Tensor 'generator/g_bn_e4_conv/Identity:0' shape=(50, 8, 8, 512) dtype=float32>, <tf.Tensor 'generator/g_bn_e5_conv/Identity:0' shape=(50, 4, 4, 512) dtype=float32>, <tf.Tensor 'generator/g_bn_e6_conv/Identity:0' shape=(50, 2, 2, 512) dtype=float32>]
first deconv g_d8_deconv g_bn_d

288 [[[176. 186. 187.]
  [177. 187. 188.]
  [178. 188. 189.]
  ...
  [187. 212. 206.]
  [187. 212. 206.]
  [191. 210. 206.]]

 [[177. 187. 188.]
  [178. 188. 189.]
  [179. 189. 190.]
  ...
  [187. 212. 206.]
  [187. 212. 206.]
  [191. 210. 206.]]

 [[179. 189. 190.]
  [179. 189. 190.]
  [179. 191. 191.]
  ...
  [190. 213. 207.]
  [189. 212. 206.]
  [191. 210. 206.]]

 ...

 [[161. 180. 174.]
  [161. 180. 174.]
  [161. 180. 174.]
  ...
  [175. 195. 186.]
  [175. 195. 186.]
  [176. 196. 187.]]

 [[161. 180. 174.]
  [161. 180. 174.]
  [161. 180. 174.]
  ...
  [175. 195. 186.]
  [176. 196. 187.]
  [176. 196. 187.]]

 [[161. 180. 174.]
  [161. 180. 174.]
  [161. 180. 174.]
  ...
  [175. 195. 186.]
  [176. 196. 187.]
  [177. 197. 188.]]]
288 [[[249. 249. 247.]
  [249. 249. 247.]
  [249. 249. 247.]
  ...
  [252. 252. 250.]
  [252. 252. 250.]
  [252. 252. 250.]]

 [[249. 249. 247.]
  [249. 249. 247.]
  [249. 249. 247.]
  ...
  [252. 252. 250.]
  [252. 252. 250.]
  [252. 252. 250.]]

 [[249. 24

288 [[[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]
  ...
  [ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]
  ...
  [ 1.  1.  1.]
  [ 1.  1.  1.]
  [ 1.  1.  1.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]
  ...
  [ 2.  2.  2.]
  [ 2.  2.  2.]
  [ 2.  2.  2.]]

 ...

 [[ 0.  1.  4.]
  [ 0.  1.  4.]
  [ 0.  0.  2.]
  ...
  [27. 28. 30.]
  [27. 28. 30.]
  [27. 28. 30.]]

 [[ 0.  1.  4.]
  [ 0.  1.  4.]
  [ 0.  0.  2.]
  ...
  [27. 28. 30.]
  [27. 28. 30.]
  [27. 28. 30.]]

 [[ 0.  1.  4.]
  [ 0.  1.  4.]
  [ 0.  0.  2.]
  ...
  [27. 28. 30.]
  [27. 28. 30.]
  [27. 28. 30.]]]
288 [[[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.

In [ ]:
arguments.lr

In [ ]:
tf.__version__